## 그룹연산

- 복잡한 데이터를 어떤 기준에 따라 여러 그룹으로 나눠서 관찰하는 것도 좋은 방법이다.
- 특정 기준을 적용하여 몇개의 그룹으로 분할하여 처리하는 것을 그룹연산이라고 부른다.
- 특히 그룹연산은 데이터를 집계, 변환, 필터링하는데 효율적이다. 데이터 그룹을 분할하는 1단계 과정은 판다스 groupby() 메소드를 사용한다.
1. 분할(split): 데이터를 특정 조건에 의해 분할
2. 적용(apply): 데이터를 집계, 변환, 필터링하는데 필요한 메소드를 적용
3. 결합(combine): 2단계의 처리 결과를 하나로 결합

### 1. 그룹 객체 만들기(분할 단계)

1개의 열을 기준으로 그룹화 : DataFrame.groupby(기준이되는열)

titanic 데이터셋에서 class열을 기준으로 하여 그룹으로 나눈다.

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
titanic = sns.load_dataset('titanic')
df = titanic.loc[:,['age','sex','class','fare','survived']]

print('승객수:', len(df))

승객수: 891


In [3]:
df.head()

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0


In [4]:
grouped = df.groupby(['class'])

df_cls 그룹객체에서 반복문을 통해 first, second, third 데이터 값의 개수를 알수있다.

In [5]:
for key, group in grouped:
    print('key:', key)
    print('number:', len(group))
    print(group.head())
    print('\n')

key: First
number: 216
     age     sex  class     fare  survived
1   38.0  female  First  71.2833         1
3   35.0  female  First  53.1000         1
6   54.0    male  First  51.8625         0
11  58.0  female  First  26.5500         1
23  28.0    male  First  35.5000         1


key: Second
number: 184
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
17   NaN    male  Second  13.0000         1
20  35.0    male  Second  26.0000         0
21  34.0    male  Second  13.0000         1


key: Third
number: 491
    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0




앞서 생성한 그룹 객체인 grouped에 연산 메소드를 적용할 수 있다. 그룹별 평균값을 구하기 위해 mean() 메소드를 적용하자.

In [6]:
average = grouped.mean()
print(average) # 범주형 컬럼은 제외

              age       fare  survived
class                                 
First   38.233441  84.154687  0.629630
Second  29.877630  20.662183  0.472826
Third   25.140620  13.675550  0.242363


그룹 객체에 get_group() 메소드를 적용하면 특정 그룹만을 선택할 수 있다. grouped 객체의 3개의 그룹 중 키 값이 'Third'인 3등석 승객의 데이터를 가진 그룹을 따로 선택해 추출해보자.

In [9]:
group3 = grouped.get_group('Third')
print(group3.head())

    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0


### 여러 열을 기준으로 그룹화

- 이번에는 groupby() 메소드에 여러 개의 열을 리스트로 전달하는 방법을 알아보자.
- 여러 개의 기준 값을 사용하기 때문에 반환되는 그룹 객체의 인덱스는 다중 구조를 갖는다.
- DataFrame.groupby(기준이되는열의 리스트)

groupby() 메소드에 두열('class', 'sex')을 인자로 전달하면 두 열이 갖는 원소 값들로 만들 수 있는 모든 조합으로 키를 생성한다. 그렇게 조합된 키를 기준으로 그룹객체를 만든다.   
class열에는 First, Second, Third라는 3개의 값이 있고, sex열에는 female, male 2개의 값이 들어 있다. 가능한 조합은('class', 'sex')형식의 투플로 지정되는대 ('First', 'female')을 포함하여 6개의 키가 조합으로 만들어진다.

In [10]:
grouped_two = df.groupby(['class','sex'])


for key, group in grouped_two:
    print('key :', key)
    print('number :', len(group))
    print(group.head())
    print('\n')

key : ('First', 'female')
number : 94
     age     sex  class      fare  survived
1   38.0  female  First   71.2833         1
3   35.0  female  First   53.1000         1
11  58.0  female  First   26.5500         1
31   NaN  female  First  146.5208         1
52  49.0  female  First   76.7292         1


key : ('First', 'male')
number : 122
     age   sex  class      fare  survived
6   54.0  male  First   51.8625         0
23  28.0  male  First   35.5000         1
27  19.0  male  First  263.0000         0
30  40.0  male  First   27.7208         0
34  28.0  male  First   82.1708         0


key : ('Second', 'female')
number : 76
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
41  27.0  female  Second  21.0000         0
43   3.0  female  Second  41.5792         1
53  29.0  female  Second  26.0000         1


key : ('Second', 'male')
number : 108
     age   sex   class  fare  survived
17   NaN  male  Second 

grouped_two 객체의 각 그룹에 mean()  메소드를 적용하면 어떻게 될까?    
이때 데이터프레임이 반환이 되는데, 키가 되는 2개의 열의 값으로 부터 2중 멀티 인덱스가 지정된다.

In [12]:
average_two = grouped_two.mean()
print(average_two)
print('\n')
print(type(average_two))

                     age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447


<class 'pandas.core.frame.DataFrame'>


전체적으로 성별에서 어느클래스든 여자가 월등히 생존율이 높다. 그리고 first클래스의 여성이 가장 생존율이 높다.   
get_group() 메소드를 활용하여 멀티 인덱스를 이용하여 특정 그룹만을 골라서 추출하는 방법이다.    
인자로 전달하는 키는 튜플로 입력한다. 예로 grouped_two 객체의 6개의 그룹 중 키가 ('Thrid', 'female')인 그룹을 추출해보자.


In [14]:
groupTf = grouped_two.get_group(('Third', 'female'))
print(groupTf)

      age     sex  class     fare  survived
2    26.0  female  Third   7.9250         1
8    27.0  female  Third  11.1333         1
10    4.0  female  Third  16.7000         1
14   14.0  female  Third   7.8542         0
18   31.0  female  Third  18.0000         0
..    ...     ...    ...      ...       ...
863   NaN  female  Third  69.5500         0
875  15.0  female  Third   7.2250         1
882  22.0  female  Third  10.5167         0
885  39.0  female  Third  29.1250         0
888   NaN  female  Third  23.4500         0

[144 rows x 5 columns]


총 144개의 데이터를 볼 수 있다.

### 2. 그룹연산 메소드(적용-결합단계)

- 앞에서 분할한 그룹 객체에 대해서 각 그룹별 평균을 계산한 것처럼, 그룹 객체에 다양한 연산을 적용할 수 있다. 이 과정을 데이터 집계(aggreagtion)라고 부른다.
- 집계 기능을 내장하고 있는 판다스 기본 함수에는 mean(), max(), min(), sum(), count(), size(), var(), std(), decribe(), info(), first(), last()등이 있다.

예로 그룹 객체의 표준편차를 구해보자 일단 판다스 객체에 std() 메소드를 사용하면 된다.

In [17]:
titatic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age','sex','class','fare','survived']]

# class열을 기준으로 분할
grouped = df.groupby(['class'])

# 그룹에 대한 연산 std() 적용
group_std_all = grouped.std()
print(group_std_all)
print(type(group_std_all))
print('\n')

# 각 그룹에 대한 fare열의 표준편차를 집계하여 시리즈로 변환
group_fare_std = grouped['fare'].std()
print(group_fare_std)
print(type(group_fare_std))

              age       fare  survived
class                                 
First   14.802856  78.380373  0.484026
Second  14.001077  13.417399  0.500623
Third   12.495398  11.778142  0.428949
<class 'pandas.core.frame.DataFrame'>


class
First     78.380373
Second    13.417399
Third     11.778142
Name: fare, dtype: float64
<class 'pandas.core.series.Series'>


집계 연산을 처리하는 사용자 정의 함수를 그룹 객체에 적용하려면 agg() 메소드를 사용한다.
다음 예제는 최대값과 최소값의 차를 계산하는 함수를 정의하여 사용한다.

In [18]:
# 그룹 객체에 agg() 메소드를 적용 - 사용자 정의 함수를 인자로 전달
def min_max(x):
    return x.max()-x.min()

# 각 그룹의 최대값과 최소값의 차이를 계산하여 그룹별로 집계
agg_minmax = grouped.agg(min_max)
print(agg_minmax)

          age      fare  survived
class                            
First   79.08  512.3292         1
Second  69.33   73.5000         1
Third   73.58   69.5500         1


/tmp/ipykernel_1130/1447101471.py:6: FutureWarning: ['sex'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  agg_minmax = grouped.agg(min_max)


위에 경고는 범주형 변수인 sex열이 집계가 되지 않았다는 경고이다. 당연히 집계는 열 데이터가 number로 되어 있는 데이터만 계산이 된다.

동시에 여러 개의 함수를 사용하여 각 그룹별 데이터에 대한 집계 연산을 처리할 수 있다. 각각의 열에 여러 개의 함수를 일괄 적용할 때는 리스트 형태로 인수를 전달하고, 열마다 다른 종류의 함수를 적용하려면 {열: 함수} 형태로 전달한다.

In [19]:
# 여러 함수를 각 열에 동일하게 적용하여 집계
agg_all = grouped.agg(['min','max'])
print(agg_all)
print('\n')

# 각 열마다 다른 함수를 적용하여 집계
agg_sep = grouped.agg({'fare':['min', 'max'], 'age':'mean'})
print(agg_sep)

         age           sex       fare           survived    
         min   max     min   max  min       max      min max
class                                                       
First   0.92  80.0  female  male  0.0  512.3292        0   1
Second  0.67  70.0  female  male  0.0   73.5000        0   1
Third   0.42  74.0  female  male  0.0   69.5500        0   1


       fare                  age
        min       max       mean
class                           
First   0.0  512.3292  38.233441
Second  0.0   73.5000  29.877630
Third   0.0   69.5500  25.140620


앞에서 2개의 함수를 리스트 형태로 입력하면 각 열에 대하여 2개의 함수의 연산 결과를 각각 집계하여 다른 열로 구분하여 표시한다. 이때 함수명을 열이름에 추가하여 2중 열 구조를 만든다.

### 그룹 연산 데이터 변환

- 앞에서 살펴본 agg() 메소드는 각 그룹별 데이터에 연산을 위한 함수를 구분 적용하고, 그룹별로 연산 결과를 집계하여 반환한다.
- transform() 메소드는 그룹별로 구분하는 각 원소에 함수를 적용하지만 그룹별 집계 대신 각 원소의 본래 행 인덱스와 열이름 기준으로 연산 결과를 반환한다. 즉, 그룹 연산의 결괄르 원본 데이터프레임과 같은 형태로 변형하여 정리하는 것이다.

예제로 'age'열에 포함된 개별 데이터의 z-score를 과정을 살펴보자.

In [20]:
# 그룹별 age열의 평균 집계 연산
age_mean = grouped.age.mean()
print(age_mean)
print('\n')

# 그룹별 age 열의 표준편차 집계 연산
age_std = grouped.age.std()
print(age_std)
print('\n')

# 그룹 객체의 age열을 iteration으로 z-score를 계산하여 출력
for key, group in grouped.age:
    group_zscore = (group- age_mean.loc[key])/age_std.loc[key] #그룹의 각 레코드를 돌면서 그에 맞는 age열의 평균값을 빼고 표준편차를 나눠준다.
    print('origin: ', key)
    print(group_zscore)
    print('\n')

class
First     38.233441
Second    29.877630
Third     25.140620
Name: age, dtype: float64


class
First     14.802856
Second    14.001077
Third     12.495398
Name: age, dtype: float64


origin:  First
1     -0.015770
3     -0.218434
6      1.065103
11     1.335321
23    -0.691315
         ...   
871    0.592221
872   -0.353543
879    1.200212
887   -1.299306
889   -0.826424
Name: age, Length: 216, dtype: float64


origin:  Second
9     -1.134029
15     1.794317
17          NaN
20     0.365855
21     0.294432
         ...   
866   -0.205529
874   -0.134106
880   -0.348375
883   -0.134106
886   -0.205529
Name: age, Length: 184, dtype: float64


origin:  Third
0     -0.251342
2      0.068776
4      0.789041
5           NaN
7     -1.851931
         ...   
882   -0.251342
884   -0.011254
885    1.109159
888         NaN
890    0.548953
Name: age, Length: 491, dtype: float64




이번에는 transform() 메소드를 사용하여 z-score를 직접 변환해보자.
z-score를 계산하는 사용자 함수를 정의하고 transform() 메소드의 인자로 전달한다. 각 그룹별 평균과 표준편차를 이용하여 각 원소의 z-score를 계산하지만, 반환되는 객체는 그룹별로 나누지 않고 원래 행 인덱스 순서로 정렬된다.

In [23]:
# zscore를 계산하는 사용자 함수 정의
def z_score(x):
    return (x- x.mean())/x.std()

# transform() 메소드를 이용하여 age열의 데이터를 zscore화
age_zscore = grouped.age.transform(z_score)
print(age_zscore.iloc[[1,9,0]]) #1,2,3그룹의 첫 데이터 확인
print('\n')
print(len(age_zscore))
print('\n')
print(age_zscore.loc[0:9]) # transform 메소드 변환 값 출력(첫10개)
print('\n')
print(type(age_zscore))

1   -0.015770
9   -1.134029
0   -0.251342
Name: age, dtype: float64


891


0   -0.251342
1   -0.015770
2    0.068776
3   -0.218434
4    0.789041
5         NaN
6    1.065103
7   -1.851931
8    0.148805
9   -1.134029
Name: age, dtype: float64


<class 'pandas.core.series.Series'>


### 그룹 객체 필터링

- 그룹 객체에 그룹객체.filter(조건식 함수) 메소드를 적용할 때 조건식을 가진 함수를 전달하면 조건이 참인 그룹만을 남긴다.


예제로는 그룹별로 데이터 개수가 200개 이상인 그룹만을 따로 필터링하는 것이다.

In [27]:
grouped_filter = grouped.filter(lambda x: len(x) >=200)
print(grouped_filter.head())
print('\n')
print(type(grouped_filter))
print('\n')
print(grouped_filter['class'].value_counts())

    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
1  38.0  female  First  71.2833         1
2  26.0  female  Third   7.9250         1
3  35.0  female  First  53.1000         1
4  35.0    male  Third   8.0500         0


<class 'pandas.core.frame.DataFrame'>


Third     491
First     216
Second      0
Name: class, dtype: int64


해당 필터링 된 데이터에서는 Third 와 First 데이터만 존재한다.

이번에는 age 열의 평균값이 30보다 작은 그룹만을 따로 선택한다.

In [29]:
age_filter = grouped.filter(lambda x: x['age'].mean() < 30)
print(age_filter.tail())
print('\n')
print(type(age_filter))
print('\n')
print(age_filter['class'].value_counts())

      age     sex   class    fare  survived
884  25.0    male   Third   7.050         0
885  39.0  female   Third  29.125         0
886  27.0    male  Second  13.000         0
888   NaN  female   Third  23.450         0
890  32.0    male   Third   7.750         0


<class 'pandas.core.frame.DataFrame'>


Third     491
Second    184
First       0
Name: class, dtype: int64


평균 나이가 30세보다 낮은 그룹은 Third ,Second 그룹이 해당이 된다.

### 그룹 객체에 함수 매핑

그룹객체.apply(매핑함수) 메소드는 판다스 객체의 개별 원소를 특정 함수를 일대일로 매핑한다. 사용자가 원하는 대부분의 연산을 그룹 객체에서 적용할 수 있다.   


class열을 기준으로 구분한 3개의 그룹에 요약통계 정보를 나타내는 describe()메소드를 적용해보자.

In [31]:
agg_grouped = grouped.apply(lambda x: x.describe())
print(agg_grouped)

                     age        fare    survived
class                                           
First  count  186.000000  216.000000  216.000000
       mean    38.233441   84.154687    0.629630
       std     14.802856   78.380373    0.484026
       min      0.920000    0.000000    0.000000
       25%     27.000000   30.923950    0.000000
       50%     37.000000   60.287500    1.000000
       75%     49.000000   93.500000    1.000000
       max     80.000000  512.329200    1.000000
Second count  173.000000  184.000000  184.000000
       mean    29.877630   20.662183    0.472826
       std     14.001077   13.417399    0.500623
       min      0.670000    0.000000    0.000000
       25%     23.000000   13.000000    0.000000
       50%     29.000000   14.250000    0.000000
       75%     36.000000   26.000000    1.000000
       max     70.000000   73.500000    1.000000
Third  count  355.000000  491.000000  491.000000
       mean    25.140620   13.675550    0.242363
       std     12.49

앞서 transform() 메소드를 사용하여 age의 열데이터에 사용자 정의함수 z-score함수를 적용한것 처럼 apply() 메소드에도 가능하다.

In [32]:
age_zscore_apply = grouped.age.apply(z_score)
print(age_zscore_apply.head())
print('\n')
print(age_zscore.head())

0   -0.251342
1   -0.015770
2    0.068776
3   -0.218434
4    0.789041
Name: age, dtype: float64


0   -0.251342
1   -0.015770
2    0.068776
3   -0.218434
4    0.789041
Name: age, dtype: float64


앞서 filter() 메서드를 사용해서 age열에 평균 30이상인 데이터를 필터링하는 것도 apply() 메소드로 적용이 가능하다.


In [33]:
age_filter_apply = grouped.apply(lambda x: x.age.mean() < 30)
print(age_filter_apply)
print('\n')

for x in age_filter_apply.index:
    if age_filter_apply[x] == True:
        age_filter_apply_df = grouped.get_group(x)
        print(age_filter_apply_df.head())
        print('\n')

class
First     False
Second     True
Third      True
dtype: bool


     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
17   NaN    male  Second  13.0000         1
20  35.0    male  Second  26.0000         0
21  34.0    male  Second  13.0000         1


    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0




filter() 메소드를 적용하면 데이터프레임이 반환이 되고, apply() 메소드를 적용하면 조건이 맞는 참 혹은 거짓 값을 나오므로 값을 보고 싶다면 반복문을 통해서 값을 출력해야한다.

## 멀티 인덱스

- groupby() 메소드에 여러 열을 리스트 형태로 전달하면 각 열들이 다중으로 행 인덱스를 구성하는 것을 보았다.
- 이처럼 판다스는 행 인덱스를 여러 레벨로 구현할 수 있도록 멀티 인덱스 클래스를 지원한다.

먼저 'class', 'sex' 열에 들어 있는 객실의 등급과 남녀 성별을 기준으로 그룹화하는 예시를 들어보자. 그리고 그룹 객체에 mean() 메소드를 적용하면 그룹별로 각 열의 평균값을 정리하여 반환한다. 이때 반환되는 객체는 멀티 인덱스를 가진 데이터프레임이다.

In [34]:
# 그룹화
grouped = df.groupby(['class','sex'])

# 그룹객체에 집계연산 mean() 적용
gdf = grouped.mean()
print(gdf)
print('\n')
print(type(gdf))

                     age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447


<class 'pandas.core.frame.DataFrame'>


데이터프레임 gdf의 멀티 인덱스에서 하나의 인덱스만 사용하는 방법을 알아보자. loc 행인덱서를 사용하여 class인덱스에서 First라는 값을 가진 행을 선택하자.

In [35]:
print(gdf.loc['First'])

              age        fare  survived
sex                                    
female  34.611765  106.125798  0.968085
male    41.281386   67.226127  0.368852


이번에는 멀티인덱스에서 두개의 인덱스를 사용하는 방법이다. loc 인덱서를 시용하고 인자로 튜플 형태로 각 인덱스에서 찾는 값을 전달한다.ㅡ

In [37]:
print(gdf.loc[('First','female')])

age          34.611765
fare        106.125798
survived      0.968085
Name: (First, female), dtype: float64


이번에는 새롭게 loc인덱서가 아닌 xs인덱서를 사용해서 인덱싱을 해보자. 다음 예제는 sex인덱스에서 male값을 갖는 행을 추출한다. 남성 승객에 한정하여 객실 등급별로 그룹을 만들어서 분석이 가능.

In [38]:
print(gdf.xs('male',level='sex'))

              age       fare  survived
class                                 
First   41.281386  67.226127  0.368852
Second  30.740707  19.741782  0.157407
Third   26.507589  12.661633  0.135447


## 피벗

- 판다스 pivot_table() 함수는 엑셀에서 사용하는 피벗테이블과 비슷한 기능을 처리한다.
- 피벗테이블을 구성하는 4가지요소 (행 인덱스, 열 인덱스, 데이터 값, 데이터 집계함수)에 적용할 데이터프레임 열을 각각 지정하여 함수의 인자로 전달한다.

다음 예제는 titanic 데이터셋을 활용하여 5개의 열을 추출하여 데이터프레임을 다시 만들고, pivot_table() 함수를 이용하여 데이터프레임의 형태를 피벗테이블로 변환한다. 5개 열 중에서 행 인덱스로 class열을 지정하고, 열 인덱스에는 sex열을 지정한다.   
age열을 피벗의 데이터 값으로 사용하고 데이터를 집계하는 함수는 평균값을 계산하는 mean 함수로 설정한다.

In [40]:
# IPyhton 디스플레이 설정 변경 
pd.set_option('display.max_columns', 10)    # 출력할 최대 열의 개수
pd.set_option('display.max_colwidth', 20)    # 출력할 열의 너비

# titanic 데이터셋에서 age, sex 등 5개 열을 선택하여 데이터프레임 만들기
titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age','sex', 'class', 'fare', 'survived']]
print(df.head())
print('\n')

    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
1  38.0  female  First  71.2833         1
2  26.0  female  Third   7.9250         1
3  35.0  female  First  53.1000         1
4  35.0    male  Third   8.0500         0




In [41]:
# 행, 열, 값, 집계에 사용할 열을 1개씩 저장
pdf1 = pd.pivot_table(df,
                      index='class',
                      columns='sex',
                      values='age',
                      aggfunc='mean',)
print(pdf1.head())

sex        female       male
class                       
First   34.611765  41.281386
Second  28.722973  30.740707
Third   21.750000  26.507589


다음 예제로는 age열 대신 survived열을 피벗의 데이터 값으로 사용하고, 데이터를 집계하는 함수를 2개를 사용하자. mean, sum함수를 리스트 형태로 입력한다. 그렇게 되면 행인덱스로는 class열의 3가지 값이 표시되고, 열 구조는 2중 멀티 인덱스가 된다.

In [42]:
pdf2 = pd.pivot_table(df,
                      index='class',
                      columns='sex',
                      values='survived',
                      aggfunc=['mean','sum'])
print(pdf2)

            mean              sum     
sex       female      male female male
class                                 
First   0.968085  0.368852     91   45
Second  0.921053  0.157407     70   17
Third   0.500000  0.135447     72   47


여기서 mean함수 계산 결과는 여자와 남자의 각각의 클래스의 생존률을 나타내고, sum은 생존자수를 나타낸다.

이제 피벗테이블 함수에 전달할 인자에 대해 각각 2개 이상의 열을 입력해보자.

In [43]:
pdf3 = pd.pivot_table(df,
                      index=['class', 'sex'],
                      columns='survived',
                      values=['age','fare'],
                      aggfunc=['mean', 'max'])
# IPython Console 디스플레이 옵션 설정
pd.set_option('display.max_columns', 10)        # 출력할 열의 개수 한도
print(pdf3.head())
print('\n')

                    mean                                      max        \
                     age                   fare               age         
survived               0          1           0           1     0     1   
class  sex                                                                
First  female  25.666667  34.939024  110.604167  105.978159  50.0  63.0   
       male    44.581967  36.248000   62.894910   74.637320  71.0  80.0   
Second female  36.000000  28.080882   18.250000   22.288989  57.0  55.0   
       male    33.369048  16.022000   19.488965   21.095100  70.0  62.0   
Third  female  23.818182  19.329787   19.773093   12.464526  48.0  63.0   

                                 
                 fare            
survived            0         1  
class  sex                       
First  female  151.55  512.3292  
       male    263.00  512.3292  
Second female   26.00   65.0000  
       male     73.50   39.0000  
Third  female   69.55   31.3875  




In [44]:
# 행, 열 구조 살펴보기
print(pdf3.index)
print(pdf3.columns)
print('\n')

MultiIndex([( 'First', 'female'),
            ( 'First',   'male'),
            ('Second', 'female'),
            ('Second',   'male'),
            ( 'Third', 'female'),
            ( 'Third',   'male')],
           names=['class', 'sex'])
MultiIndex([('mean',  'age', 0),
            ('mean',  'age', 1),
            ('mean', 'fare', 0),
            ('mean', 'fare', 1),
            ( 'max',  'age', 0),
            ( 'max',  'age', 1),
            ( 'max', 'fare', 0),
            ( 'max', 'fare', 1)],
           names=[None, None, 'survived'])




행 인덱스와 데이터값에 해당하는 열을 각각 2개 지정하고, 열 위치에 해당하는 열은 1개를 지정했다. 또한 집계함수를 2개 지정했다. 데이터프레임 pdf3 index속성과 columns속성을 출력한 값을 보았을때 행인덱스는 (class, sex) 2개의 열이 사용되어 2중 인덱스 구조이다. 그리고 열은 3중 구조이다. 데이터 집계함수인 mean과 max가 한층을 이루고 데이터 값을 구분하는 age, fare열이 다음 층을 이룬다. 마지막으로 구조 여부를 나타내는 survived열이 위치한다.

이제 pdf3의 행을 선택하기 위해 xs 인덱서를 사용하는 방법을 익혀보자. xs 인덱서는 기본값으로 행 인덱스에 접근하고, 축 값은 axis=0으로 자동 설정된다. 먼저 First승객 데이터를 추출해보자.

In [45]:
print(pdf3.xs('First'))

               mean                                      max                \
                age                   fare               age          fare   
survived          0          1           0           1     0     1       0   
sex                                                                          
female    25.666667  34.939024  110.604167  105.978159  50.0  63.0  151.55   
male      44.581967  36.248000   62.894910   74.637320  71.0  80.0  263.00   

                    
                    
survived         1  
sex                 
female    512.3292  
male      512.3292  


다음으로 행인덱스 레벨 0에서 First를 가져오고 행 인덱스는 female을 가져와서 해당하는 데이터를 추출해보자.

In [48]:
print(pdf3.xs(('First','female')))

            survived
mean  age   0            25.666667
            1            34.939024
      fare  0           110.604167
            1           105.978159
max   age   0            50.000000
            1            63.000000
      fare  0           151.550000
            1           512.329200
Name: (First, female), dtype: float64


이번에는 행 인덱스 레벨을 직접 지정하는 방법을 익혀보자. sex 레벨에서 남성 승객을 나타내는 male에 해당하는 데이터만 추출해보자.

In [49]:
print(pdf3.xs('male',level='sex'))

               mean                                    max                \
                age                  fare              age          fare   
survived          0          1          0          1     0     1       0   
class                                                                      
First     44.581967  36.248000  62.894910  74.637320  71.0  80.0  263.00   
Second    33.369048  16.022000  19.488965  21.095100  70.0  62.0   73.50   
Third     27.255814  22.274211  12.204469  15.579696  74.0  45.0   69.55   

                    
                    
survived         1  
class               
First     512.3292  
Second     39.0000  
Third      56.4958  


행 인덱스 레벨 0에서 Second를 가져오고, 행 인덱스 레벨 sex에서 male을 가져와보자.

In [50]:
print(pdf3.xs(('Second','male'), level=[0,'sex'])) # 레벨을 0 또는 'class'로 숫자형 혹은 이름으로 레벨지정이 가능하다.

                  mean                               max                  
                   age               fare            age        fare      
survived             0       1          0        1     0     1     0     1
class  sex                                                                
Second male  33.369048  16.022  19.488965  21.0951  70.0  62.0  73.5  39.0


이제 열 접근을 해보자. xs 인덱서는 기본 설정이 axis=0으로 되어있다고 앞서 서술했으니 열을 추출하려면 axis=1로 설정해야한다.   
먼저 pdf3의 mean열을 평균값 데이터를 추출해보자.

In [52]:
print(pdf3.xs('mean',axis=1))

                     age                   fare            
survived               0          1           0           1
class  sex                                                 
First  female  25.666667  34.939024  110.604167  105.978159
       male    44.581967  36.248000   62.894910   74.637320
Second female  36.000000  28.080882   18.250000   22.288989
       male    33.369048  16.022000   19.488965   21.095100
Third  female  23.818182  19.329787   19.773093   12.464526
       male    27.255814  22.274211   12.204469   15.579696


다음으로 열 인덱스 레벨 0에서 평균값을 나타내는 mean을 지정하고, 열 인덱스 레벨 1에서 나이를 나타내는 age를 지정해보자.

In [54]:
print(pdf3.xs(('mean','age'),axis=1, level=[0,1]))

survived               0          1
class  sex                         
First  female  25.666667  34.939024
       male    44.581967  36.248000
Second female  36.000000  28.080882
       male    33.369048  16.022000
Third  female  23.818182  19.329787
       male    27.255814  22.274211


열 인덱스 레벨을 직접 지정할 수 있다. survived 레벨의 값이 1에 해당하는 데이터만 추출한다.

In [55]:
print(pdf3.xs(1,level='survived', axis=1))

                    mean               max          
                     age        fare   age      fare
class  sex                                          
First  female  34.939024  105.978159  63.0  512.3292
       male    36.248000   74.637320  80.0  512.3292
Second female  28.080882   22.288989  55.0   65.0000
       male    16.022000   21.095100  62.0   39.0000
Third  female  19.329787   12.464526  63.0   31.3875
       male    22.274211   15.579696  45.0   56.4958


마지막으로 열 인덱스 레벨에서 최대값을 나타내는 max를 가져오고, 레벨 1에서 객실 요금을 나타내는 fare를 가져온다. survived 열을 나타내는 레벨 2에서 구조박지 못한 승객을 나타내는 0을 가져온다. 이때 반환되는 데이터는 구조 받지 못한 승객들의 요금 최대값이다.

In [56]:
print(pdf3.xs(('max', 'fare', 0),
              level=[0,1,2], axis=1))

                  max
                 fare
survived            0
class  sex           
First  female  151.55
       male    263.00
Second female   26.00
       male     73.50
Third  female   69.55
       male     69.55
